In [ ]:
import nltk
import csv
from sklearn import preprocessing
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import ( BernoulliNB,ComplementNB,MultinomialNB)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import metrics

nltk.download(["names","stopwords","averaged_perceptron_tagger","vader_lexicon","punkt"])

In [ ]:
tweets = []
testTweets = []
#Data understanding & reading
with open('dreaddit-train.csv',encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:
            tweets.append([row[0],row[3],row[5]])
        line_count += 1
    print(f'Processed {line_count} lines for training.')
with open('dreaddit-test.csv',encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:
            testTweets.append([row[1],row[4],row[5]])
        line_count += 1
    print(f'Processed {line_count} lines for testing.')

In [ ]:
i = 0
added = 0
subreddits = {}
sia = SentimentIntensityAnalyzer()
numericElements = []
numericElements2 = []
labels = []
for tweet in tweets:
    polarity = sia.polarity_scores(tweet[1])
    if tweet[0] not in subreddits:
        subreddits[tweet[0]] = added
        added = added + 1
    numericElements.append([polarity["compound"]])
    #del(polarity["compound"])
    numericElements2.append({"compound":polarity["compound"]})
    i = i + 1
    labels.append(tweet[2])
print(numericElements[0])
print(subreddits)
print(numericElements2[0])

In [ ]:
numericElementsTest = []
numericElementsTest2 = []
labelsTest = []
for tweet in testTweets:
    polarity = sia.polarity_scores(tweet[1])
    numericElementsTest.append([polarity["compound"]])
    labelsTest.append(tweet[2])
    #del(polarity["compound"])
    numericElementsTest2.append({"compound":polarity["compound"]})
print(numericElementsTest[0])

In [ ]:
model = KNeighborsClassifier(n_neighbors=23)
scaler = preprocessing.MinMaxScaler()
scaled_values = scaler.fit_transform(numericElements) 
model.fit(scaled_values,labels)
scaled_values_test = scaler.fit_transform(numericElementsTest) 
y_test = model.predict(scaled_values_test)
print("Accuracy:",metrics.accuracy_score(y_test, labelsTest))

In [ ]:
i = 0
for tweet in tweets:
    catDictionary = {}
    for cat in subreddits.keys():
        sameCat = cat == tweet[0]
        if sameCat:
            numericElements[i].append(1)
            catDictionary[cat] = 1
        else :
            numericElements[i].append(0)
            catDictionary[cat] = 0
    numericElements2[i] = (dict(catDictionary, **numericElements2[i]),tweet[2])
    i = i + 1
print(numericElements[0])

In [ ]:
i = 0
for tweet in testTweets:
    catDictionary = {}
    for cat in subreddits.keys():
        sameCat = cat == tweet[0]
        if sameCat:
            numericElementsTest[i].append(1)
            catDictionary[cat] = 1
        else :
            numericElementsTest[i].append(0)
            catDictionary[cat] = 0
    numericElementsTest2[i] = (dict(catDictionary,**numericElementsTest2[i]),tweet[2])
    i = i + 1
print(numericElementsTest[0])

In [ ]:
model = KNeighborsClassifier(n_neighbors=23)
scaler = preprocessing.MinMaxScaler()
scaled_values = scaler.fit_transform(numericElements) 
model.fit(scaled_values,labels)

In [ ]:
scaled_values_test = scaler.fit_transform(numericElementsTest) 
y_test = model.predict(scaled_values_test)
print("Accuracy:",metrics.accuracy_score(y_test, labelsTest))

In [ ]:
classifiers = {
    "BernoulliNB": BernoulliNB(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "AdaBoostClassifier": AdaBoostClassifier(),
}
for name, sklearn_classifier in classifiers.items():
    classifier = nltk.classify.SklearnClassifier(sklearn_classifier)
    classifier.train(numericElements2)
    accuracy = nltk.classify.accuracy(classifier, numericElementsTest2)
    print(F"{accuracy:.2%} - {name}")